In [ ]:
import glob
import mne 
import pandas as pd
import numpy as np
import seaborn as sns
import pingouin as pg

In [ ]:
def get_files(subjs, exclude):
    
    for i in range(len(exclude)):
        exclude[i] = f'./analysis/Rise00{exclude[i]}-ave.fif'
    
    subjs = list(set(subjs) - set(exclude))
    
    return subjs

In [ ]:
def create_df(subjs, events_id, ch, time_win, peak_func):

    data_Pz = []

    for subj in subjs:
        for event_name in events_id:
            evk = mne.read_evokeds(subj, condition=event_name, verbose=False)

            evk.pick(ch)

            data = evk.data[0][time_win[0]:time_win[1]]

            if peak_func == 'auc':
                data_Pz.append(np.sum(np.abs(data)))
            elif peak_func == 'peak':
                data[data < 0] = 0
                data_Pz.append(np.amax(data))

    subjs_for_df = sorted([int(name[17: 19]) for name in subjs])


    df = pd.DataFrame({
            'Subject': np.repeat(subjs_for_df, len(events_id)),
            'Condition': np.tile(['Neutral', 'Rise', 'Fall'], len(subjs_for_df)),
            'uV': data_Pz
        })

    return df

In [ ]:
evokeds_files = glob.glob('./analysis/Rise*-ave.fif')

subjs_all = get_files(evokeds_files, exclude=[])
subjs_excl = get_files(evokeds_files, exclude=['2', '3', '22', '25', '33', '34', '36'])

df = create_df  (
                    subjs_all, 
                    ['1002', '1003', '1004'], 
                    'Pz', 
                    [220, 350], 
                    'auc'
                )

In [ ]:
ax = sns.lineplot(x='Condition', y = 'uV', units= "Subject", estimator=None, data = df, alpha=0.3, hue="Subject", legend=False)
sns.stripplot(x="Condition", y="uV", hue="Condition", data=df, jitter=False, alpha =0.3)
ax = sns.pointplot(x='Condition', y = 'uV', data = df)
ax.set_title('CPz AUC between Conditions w Subj. Excl.')
sns.despine()

In [ ]:
# Repeated measure ANOVA
pg.rm_anova(dv='uV', 
               within='Condition', 
               subject="Subject",
               data=df,
               detailed=True)

In [ ]:
# Paired t-test between Rise & Neutral
pg.ttest(x=df[df.Condition=='Rise'].uV, y=df[df.Condition=='Neutral'].uV, paired=True)

In [ ]:
# Paired t-test between Rise & Fall
pg.ttest(x=df[df.Condition=='Rise'].uV, y=df[df.Condition=='Fall'].uV, paired=True)